In [1]:

!pip install import_ipynb
import import_ipynb
from google.colab import drive
drive.mount('/content/gdrive',  force_remount=True)

% cd /content/gdrive/My Drive/Project\ FPAR2/ego-rnn

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=eb5b2a5085f309b8670f9c6bbcd7de1ba012055134d6f6c434fd406cfa5443c7
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb
Mounted at /content/gdrive
/content/gdrive/My Drive/Project FPAR2/ego-rnn


In [3]:
import torch
import resnetMod
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from MyConvLSTMCell import *

class MSHead(nn.Module): 
    def __init__(self):
        super(MSHead, self).__init__()
        self.relu=nn.ReLU(inplace=False)
        self.convol2=torch.nn.Conv2d(in_channels=512, out_channels=100, kernel_size=1, stride=1, padding=0)
        # bzx100x7x7
        self.supm=nn.Linear(100*7*7,7*7) #bzx2*7*7
        torch.nn.init.xavier_uniform(self.supm.weight)
        torch.nn.init.xavier_uniform(self.convol2.weight)
        self.sigm=nn.Sigmoid()
        #self.softmax = nn.Softmax()

    def forward(self, x):
        x = self.relu(x)
        x = self.convol2(x)
        x = x.view(x.size(0), -1)
        x = self.supm(x)
        x = x.view(x.size(0),7,7) #It could be also (x.size(0), 2,7,7) in another implementation
        x=self.sigm(x)
        #x=self.softmax(x, dim=1) #I don't need it
        return x
        #dovrebbe tornare (32,7*7)

class CNNTrunk(nn.Module):
    def __init__(self, num_classes=61, mem_size=512):
        super(CNNTrunk, self).__init__()
        self.num_classes = num_classes
        self.resNet = resnetMod.resnet34(True, True)
        self.mem_size = mem_size
        self.weight_softmax = self.resNet.fc.weight
        self.lstm_cell = MyConvLSTMCell(512, mem_size)
        self.avgpool = nn.AvgPool2d(7)
        self.dropout = nn.Dropout(0.7)
        self.fc = nn.Linear(mem_size, self.num_classes)
        self.classifier = nn.Sequential(self.dropout, self.fc)

    def forward(self, inputVariable):
        state = (Variable(torch.zeros((inputVariable.size(1), self.mem_size, 7, 7)).cuda()),
                 Variable(torch.zeros((inputVariable.size(1), self.mem_size, 7, 7)).cuda()))
        idts=[]
        for t in range(inputVariable.size(0)):
            logit, feature_conv, feature_convNBN = self.resNet(inputVariable[t]) 
            #logit sono i pesi delle uscite dei 1000 risultati di output, feature_conv contiene la matrice dell'activation map
            #dell'ultimo strato convoluzionale, feature_convNBN quest'ultima a cui non è stata applicata la normalizzazione
            bz, nc, h, w = feature_conv.size() #bz sarebbe il numero delle immagini, poi c'è number of channels che dovrebbe
                                                #essere 512
            idts.append(feature_conv)
            feature_conv1 = feature_conv.view(bz, nc, h*w)
            probs, idxs = logit.sort(1, True) #Qua sta sortando gli indici per numero maggiore: vuole vedere quale è l'output più sicuro
            class_idx = idxs[:, 0]
            cam = torch.bmm(self.weight_softmax[class_idx].unsqueeze(1), feature_conv1) 
            #sta moltiplicando l'array delle classi più probabili per l'activation cam
            attentionMAP = F.softmax(cam.squeeze(1), dim=1)
            attentionMAP = attentionMAP.view(attentionMAP.size(0), 1, 7, 7)
            attentionFeat = feature_convNBN * attentionMAP.expand_as(feature_conv)
            #ora metto feature_conv perché non voglio mettere dentro le CAM
            state = self.lstm_cell(attentionFeat, state)
        feats1 = self.avgpool(state[1]).view(state[1].size(0), -1)
        feats = self.classifier(feats1)
        return feats, feats1, idts



In [5]:
from __future__ import print_function, division
from objectAttentionModelConvLSTM import *
!pip install import_ipynb
import import_ipynb

import os
import torch
from torch.utils.data import Dataset
from PIL import Image
import numpy as np
import glob
import random
from google.colab import drive
from PIL import Image
from torch.nn import functional as F

from spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                RandomHorizontalFlip)
!pip install tensorboardX
from tensorboardX import SummaryWriter

import argparse
import sys

from makeDatasetRGB import *

% cd /content/gdrive/My Drive/Project FPAR2/ego-rnn

train_data_dir='/content/gdrive/My Drive/Project FPAR2/ego-rnn/GTEA61/processed_frames2'
val_data_dir='/content/gdrive/My Drive/Project FPAR2/ego-rnn/GTEA61/processed_frames2'

out_dir='experiments/sparnet'
seqLen=7
trainBatchSize=32
valBatchSize=64
numEpochs=200
lr1=1e-4
decay_factor=0.1
decay_step=[25,75,150]
memSize=512

print('Loading data')
normalize = Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
spatial_transform = Compose([Scale(256), RandomHorizontalFlip(), MultiScaleCornerCrop([1, 0.875, 0.75, 0.65625], 224),
                                 ToTensor(), normalize])

vid_seq_train = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=seqLen, fmt='.png', train=True)

train_loader = torch.utils.data.DataLoader(vid_seq_train, batch_size=trainBatchSize,
                            shuffle=False, num_workers=3, pin_memory=True)
    
print('Done data loader!')  

vid_seq_val = makeDataset(train_data_dir, spatial_transform=Compose([Scale(256), CenterCrop(224), ToTensor(), normalize]), seqLen=seqLen, fmt='.png', train=False)

val_loader = torch.utils.data.DataLoader(vid_seq_val, batch_size=trainBatchSize,
                            shuffle=False, num_workers=3, pin_memory=True)

#Data loader
print('Loading data MAP')

spatial_transform=Compose([Scale(256), CenterCrop(224), Scale(size=(7,7)), ToTensor()])

train_loader_m = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=seqLen, fmt='.png', train=True, frameType='m')

train_loader_map = torch.utils.data.DataLoader(train_loader_m, batch_size=trainBatchSize,
                            shuffle=False, num_workers=3, pin_memory=True)

val_loader_m = makeDataset(train_data_dir, spatial_transform=spatial_transform, seqLen=seqLen, fmt='.png', train=False, frameType='m')

val_loader_map = torch.utils.data.DataLoader(val_loader_m, batch_size=trainBatchSize,
                            shuffle=False, num_workers=3, pin_memory=True)


torch.autograd.set_detect_anomaly(True)



/content/gdrive/My Drive/Project FPAR2/ego-rnn
Loading data
Done data loader!
Loading data MAP


In [ ]:
for t in train_loader_map:
  print(t[0].size())

[Errno 2] No such file or directory: './GTEA61/processed_frames2/S1'
/content/gdrive/.shortcut-targets-by-id/1LJLFVWOiRrTlmqPhKcItrvgalbU94D5z/Project FPAR/ego-rnn
 attentionMapModel.ipynb	  main-run-twoStream.ipynb
 Checkdatabases.ipynb		  makeDatasetFlow.ipynb
 CNNTrunk.ipynb			  MyConvLSTMCell.ipynb
'Copia di main_run_rgb.ipynb'	  objectAttentionModelConvLSTM.ipynb
 eval-run-flow.ipynb		  plotting.ipynb
 eval-run-rgb.ipynb		  prepareGTEA61Dataset.ipynb
 eval-run-twoStream.ipynb	  __pycache__
 eval-run-twoStream-joint.ipynb   README.md
 experiments			  resnetMod.ipynb
 experiments3			  results
 flow				  sparnet2.ipynb
 flow_resnet.ipynb		  sparnet.ipynb
 genAttentionMap.ipynb		  spatial_transforms.ipynb
 GTEA61				  Tabella.gsheet
 main-run-flow.ipynb		  TrialVal.ipynb
 main_run_rgb.ipynb		  twoStreamModel.ipynb


In [7]:
numEpochs=150
num_classes=61
seqLen=7
trainBatchSize=32
valBatchSize=64
numEpochs=150
lr1=1e-3
decay_factor=0.1
decay_step=[25,75]
memSize=512

torch.autograd.set_detect_anomaly(True)

path_rgb='/content/gdrive/My Drive/Project FPAR2/ego-rnn/experiments/rgb/model_rgb_state_dict.pth'

model = CNNTrunk(num_classes=num_classes, mem_size=memSize).cuda()

model.load_state_dict(torch.load(path_rgb), strict=False)
ms=MSHead().cuda()

train_params = []

for params in ms.supm.parameters():
    params.requires_grad = True
    train_params += [params]

for params in ms.convol2.parameters():
    params.requires_grad = True
    train_params += [params]

for params in model.parameters():
    params.requires_grad = False

for params in model.resNet.layer4[0].conv1.parameters():
    params.requires_grad = True
    train_params += [params]

for params in model.resNet.layer4[0].conv2.parameters():
    params.requires_grad = True
    train_params += [params]

for params in model.resNet.layer4[1].conv1.parameters():
    params.requires_grad = True
    train_params += [params]

for params in model.resNet.layer4[1].conv2.parameters():
    params.requires_grad = True
    train_params += [params]

for params in model.resNet.layer4[2].conv1.parameters():
    params.requires_grad = True
    train_params += [params]
        #
for params in model.resNet.layer4[2].conv2.parameters():
    params.requires_grad = True
    train_params += [params]
        #
for params in model.resNet.fc.parameters():
    params.requires_grad = True
    train_params += [params]

ms.convol2.train(True)
ms.supm.train(True)

model.resNet.layer4[0].conv1.train(True)
model.resNet.layer4[0].conv2.train(True)
model.resNet.layer4[1].conv1.train(True)
model.resNet.layer4[1].conv2.train(True)
model.resNet.layer4[2].conv1.train(True)
model.resNet.layer4[2].conv2.train(True)

for params in model.lstm_cell.parameters():
    params.requires_grad = True
    train_params += [params]

for params in model.classifier.parameters():
    params.requires_grad = True
    train_params += [params]

model.lstm_cell.train(True)

model.classifier.train(True)
model.resNet.fc.train(True)
model.classifier.train(True)
model.cuda()


loss_fn = nn.CrossEntropyLoss()
loss_fn_binary=nn.BCELoss()

optimizer_fn = torch.optim.Adam(train_params, lr=lr1, weight_decay=4e-5, eps=1e-4)
optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer_fn, milestones=decay_step,
                                                           gamma=decay_factor)
  
loss_ms=[]
loss_classifier=[]
accuracy_ms=[]
accuracy_classifier=[]

loss_ms_val=[]
loss_classifier_val=[]
accuracy_ms_val=[]
accuracy_classifier_val=[]

min_accuracy = 0

for epoch in range(numEpochs):

    optim_scheduler.step()
    epoch_loss = 0
    numCorrTrain = 0
    trainSamples = 0
    iter=0

    numCorrTrain_ms=0
    trainSamples_ms=0

    model.lstm_cell.train(True)
    model.classifier.train(True)
    model.resNet.layer4[0].conv1.train(True)
    model.resNet.layer4[0].conv2.train(True)
    model.resNet.layer4[1].conv1.train(True)
    model.resNet.layer4[1].conv2.train(True)
    model.resNet.layer4[2].conv1.train(True)
    model.resNet.layer4[2].conv2.train(True)
    model.resNet.fc.train(True)

    print(f'Epoch {epoch}')
    
    loss_ms_epoch=0
    loss_classifier_epoch=0 

    for train_image,idts_image in zip(train_loader, train_loader_map): #It was inputs, targets

        iter+=1
        input_train=train_image[0]
        trainSamples+=input_train.size(0)
        input_train=Variable(input_train.permute(1,0,2,3,4).to(device='cuda'))
        
        target_label=Variable(train_image[1]).to(device='cuda')
        idts_labels=Variable(idts_image[0].squeeze().permute(1,0,2,3).to(device='cuda'))
        optimizer_fn.zero_grad()
        output_label, f, idts = model(input_train) 
        idts_built=[]

        for id in idts:
          idts_built.append(ms(id.cuda()))

        #Now trasforming idts_labels into arrays of 7,32,49
        idts_labels=(idts_labels>0.05).float().view(idts_labels.size(0),idts_labels.size(1),-1)

        #Computing loss
        loss_ms_local=0
        loss_classifier_local=0

        idts_built=torch.stack(idts_built)
        idts_built=idts_built.view(idts_built.size(0),idts_built.size(1),-1)

        for i in range(idts_labels.size(0)):
            loss_ms_local+=loss_fn_binary(idts_built[i], idts_labels[i])

        idts_built_bin=(idts_built>0.5).int()
        
        numCorrTrain_ms+=(idts_built_bin==idts_labels.int()).sum().item()
        trainSamples_ms+=torch.ones(idts_built.size()).sum().item()

        loss_classifier_local=loss_fn(output_label, target_label)

        loss_ms_epoch+=loss_ms_local.item()
        loss_classifier_epoch+=loss_classifier_local.item()

        loss=loss_ms_local+loss_classifier_local*5
        loss.backward()

        #Computing accuracy

        _, predicted = torch.max(output_label.data, 1)
        numCorrTrain += (predicted == target_label).sum().item()
        optimizer_fn.step()
        

    print('Ms task loss')
    print(loss_ms_epoch/iter)
    print('Accuracy Ms task')
    print(numCorrTrain_ms/trainSamples_ms)


    print('Local classifier loss')
    print(loss_classifier_epoch/iter)
    print('Accuracy classifier')
    print(numCorrTrain/trainSamples)

    loss_ms.append(loss_ms_epoch/iter)
    loss_classifier.append(loss_classifier_epoch/iter)
    accuracy_ms.append(numCorrTrain_ms/trainSamples_ms)
    accuracy_classifier.append(numCorrTrain/trainSamples)

    model.train(False)

    #VALIDATION

    epoch_loss = 0
    numCorrTrain = 0
    trainSamples = 0
    iter=0
    numCorrTrain_ms=0
    trainSamples_ms=0
    
    loss_ms_epoch=0
    loss_classifier_epoch=0 

    for train_image,idts_image in zip(val_loader, val_loader_map): #It was inputs, targets

        iter+=1
        input_train=train_image[0]
        trainSamples+=input_train.size(0)
        input_train=Variable(input_train.permute(1,0,2,3,4).to(device='cuda'))

        target_label=Variable(train_image[1]).to(device='cuda')
        idts_labels=Variable(idts_image[0].squeeze().permute(1,0,2,3).to(device='cuda'))
        

        output_label, f, idts = model(input_train) 
        idts_built=[]

        for id in idts:
          idts_built.append(ms(id.cuda()))


        idts_labels=(idts_labels>0.05).float().view(idts_labels.size(0),idts_labels.size(1),-1)

        #Computing loss
        loss_ms_local=0
        loss_classifier_local=0

        idts_built=torch.stack(idts_built)
        idts_built=idts_built.view(idts_built.size(0),idts_built.size(1),-1)

        for i in range(idts_labels.size(0)):
            loss_ms_local+=loss_fn_binary(idts_built[i], idts_labels[i])

        #Is this a good choice? Can I use binary cross entropy without this?
        idts_built_bin=(idts_built>0.5).int()
        
        numCorrTrain_ms+=(idts_built_bin==idts_labels.int()).sum().item()
        trainSamples_ms+=torch.ones(idts_built.size()).sum().item()

        loss_classifier_local=loss_fn(output_label, target_label)

        loss_ms_epoch+=loss_ms_local.item()
        loss_classifier_epoch+=loss_classifier_local.item()        

        #Computing accuracy

        _, predicted = torch.max(output_label.data, 1)
        numCorrTrain += (predicted == target_label).sum().item()
        

    print('Ms task loss val')
    print(loss_ms_epoch/iter)
    print('Accuracy Ms task val')
    print(numCorrTrain_ms/trainSamples_ms)


    print('Local classifier loss val')
    print(loss_classifier_epoch/iter)
    print('Accuracy classifier val')
    print(numCorrTrain/trainSamples)

    loss_ms_val.append(loss_ms_epoch/iter)
    loss_classifier_val.append(loss_classifier_epoch/iter)
    accuracy_ms_val.append(numCorrTrain_ms/trainSamples_ms)
    accuracy_classifier_val.append(numCorrTrain/trainSamples)


import matplotlib.pyplot as plt

plt.figure()
plt.plot([x for x in range(len(loss_ms))], loss_ms, 'r', label='Loss ms')
plt.plot([x for x in range(len(loss_ms_val))], loss_ms_val, 'b', label='Loss ms val')

plt.figure()
plt.plot([x for x in range(len(loss_classifier))], loss_classifier, 'r', label='Loss classifier')
plt.plot([x for x in range(len(loss_classifier_val))], loss_classifier_val, 'b', label='Loss classifier val')

plt.figure()
plt.plot([x for x in range(len(accuracy_ms))], accuracy_ms, 'r', label='accuracy_ms')
plt.plot([x for x in range(len(accuracy_ms_val))], accuracy_ms_val, 'b', label='accuracy_ms_val')

plt.figure()
plt.plot([x for x in range(len(accuracy_classifier))], accuracy_classifier, 'r', label='accuracy_classifier')
plt.plot([x for x in range(len(accuracy_classifier_val))], accuracy_classifier_val, 'b', label='accuracy_classifier_val')

    

MyConvLSTMCell.ipynb:31: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTMCell.ipynb:32: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvLSTMCell.ipynb:33: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTMCell.ipynb:35: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTMCell.ipynb:36: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvLSTMCell.ipynb:37: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTMCell.ipynb:39: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyConvLSTMCell.ipynb:40: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
MyConvLSTMCell.ipynb:41: UserWarning: nn.init.xavier_normal is now deprecated in favor of nn.init.xavier_normal_.
MyCo

Epoch 0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  from ipykernel import kernelapp as app
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  app.launch_new_instance()
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1639: 

Ms task loss
6.639254700053822
Accuracy Ms task
0.6850713473491618
Local classifier loss
3.894691142168912
Accuracy classifier
0.09970674486803519
Ms task loss val
6.629225492477417
Accuracy Ms task val
0.6993817231326028
Local classifier loss val
3.5495481491088867
Accuracy classifier val
0.12931034482758622
Epoch 1
Ms task loss
5.18605240908536
Accuracy Ms task
0.6975453775980438
Local classifier loss
3.826653545553034
Accuracy classifier
0.07917888563049853
Ms task loss val
4.665336012840271
Accuracy Ms task val
0.7030009047954157
Local classifier loss val
3.5898911356925964
Accuracy classifier val
0.09482758620689655
Epoch 2
Ms task loss
3.8724713758988814
Accuracy Ms task
0.7360703812316716
Local classifier loss
3.864634015343406
Accuracy classifier
0.05571847507331378
Ms task loss val
4.52667111158371
Accuracy Ms task val
0.7040313662410778
Local classifier loss val
3.4902750849723816
Accuracy classifier val
0.09482758620689655
Epoch 3
Ms task loss
3.7533020322973076
Accuracy Ms 

KeyboardInterrupt: ignored

In [ ]:
!pip install import_ipynb
import import_ipynb
from google.colab import drive
% cd /content/gdrive/My Drive/Project\ FPAR/ego-rnn

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from PIL import Image
import torch

import torch.nn as nn
from torch.nn import functional as F
from torch.autograd import Variable
from torchvision import datasets, transforms
#from spatial_transforms import (Compose, ToTensor, CenterCrop, Scale, Normalize, MultiScaleCornerCrop,
                                 #RandomHorizontalFlip)

idt=torch.rand(3,3,3)
id=(torch.rand(3,3,3)>0.5).float()
nn.

'''
p='/content/gdrive/My Drive/Project FPAR/ego-rnn/GTEA61/processed_frames/S3/take_water/3/mmaps/map0020.png'
img = Image.open(p)
display(img)
spatial_transform=Compose([Scale(256), CenterCrop(224), ToTensor()])
img=spatial_transform(img.convert('RGB'))
print(img)

img=img.unsqueeze(0)
img=F.interpolate(img, size=(7, 7), mode='bilinear')
img=torch.mean(img, [0,1])
print(img.size())
'''
print(o.size())

[Errno 2] No such file or directory: '/content/gdrive/My Drive/Project FPAR/ego-rnn'
/content
torch.Size([2, 3, 3, 3])


In [ ]:
w=torch.tensor([0.4, 0.8])
idt=torch.rand(2)
id=torch.empty(2).random_(2)
ll=nn.BCEWithLogitsLoss(weight=w)
cr=ll(idt,id)
print(cr)

tensor(0.5860)
